In [1]:
import sys
sys.path.append("..")
import msgspec
import faker
from tqdm.auto import tqdm
from typing import Any, cast
import pickle
import gc
import random


In [2]:
fake = faker.Faker("ru_RU")

In [3]:
from dkvs.bptree import BPTree
BPTree.HALF_INODE_SIZE = 32
BPTree.MAX_INODE_SIZE = BPTree.HALF_INODE_SIZE * 2
BPTree.HALF_LEAF_SIZE = 128
BPTree.MAX_LEAF_SIZE = BPTree.HALF_LEAF_SIZE * 2

In [4]:
class MyKey(msgspec.Struct, frozen=True, order=True):
    full_name: str

class Data(msgspec.Struct):
    data: Any

In [5]:
index = BPTree[MyKey, Data]()

In [6]:
faker.Faker.seed(1)
fake = faker.Faker("ru_RU")

In [7]:
data = []
gc.collect()

340

In [8]:
%%time
for i in tqdm(range(10_000_000)):
    random_int = fake.random_int(0, 99_999_999)
    val = f"{random_int:010} " + fake.name()
    result = index.insert(MyKey(val))
    if result[0] is None:
        print(f"Duplicate key: {val}: {result[1]}")
    data.append(val)

  0%|          | 0/10000000 [00:00<?, ?it/s]

CPU times: user 9min 57s, sys: 10.3 s, total: 10min 7s
Wall time: 10min 3s


In [9]:
index.validate(), gc.collect()

({}, 18)

In [10]:
index.find(MyKey('xxx'))

(None, None, None)

In [19]:
%%timeit
index.find(MyKey('0001677529 Шаров Леонтий Филатович'))

5.66 µs ± 87.8 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [12]:
index.print_node(index.tree[index.root_node])


node-idx:3887, prev:None, next:None, level: 3  <============ *ROOT*
         |-> MyKey(full_name='0003669152 Фадеев Мечислав Демидович'): 68, 50389
         |-> MyKey(full_name='0006732412 Субботина Мария Федоровна'): 50389, 26997
         |-> MyKey(full_name='0009781365 Мария Юрьевна Третьякова'): 26997, 58479
         |-> MyKey(full_name='0012902097 Антип Филиппович Савин'): 58479, 14132
         |-> MyKey(full_name='0015841405 Епифан Абрамович Самойлов'): 14132, 58390
         |-> MyKey(full_name='0018982670 Антонина Юрьевна Некрасова'): 58390, 28394
         |-> MyKey(full_name='0021751907 Чернова Ираида Сергеевна'): 28394, 58048
         |-> MyKey(full_name='0024692102 Лукьян Федосеевич Ершов'): 58048, 7059
         |-> MyKey(full_name='0027870494 Кононова Варвара Наумовна'): 7059, 53046
         |-> MyKey(full_name='0031125905 Надежда Викторовна Шилова'): 53046, 26795
         |-> MyKey(full_name='0034584616 Якушев Степан Богданович'): 26795, 50913
         |-> MyKey(full_name='

In [13]:
index.min()[1]

MyKey(full_name='0000000000 Максимова Марфа Альбертовна')

In [14]:
index.max()[1]

MyKey(full_name='0099999997 Антонина Леоновна Костина')

In [15]:
data[0:10]

['0018034063 Брагина Ангелина Ждановна',
 '0015826780 тов. Воронов Натан Глебович',
 '0012597620 Никита Феликсович Кошелев',
 '0000282669 Федотова Елизавета Эдуардовна',
 '0013720696 Алевтина Архиповна Козлова',
 '0087180606 Субботина Марина Даниловна',
 '0056655527 Лобанова Елена Павловна',
 '0066546792 Коновалова Елизавета Дмитриевна',
 '0061686932 Акулина Николаевна Дьячкова',
 '0074686034 Антонина Геннадиевна Евдокимова']

In [16]:
random.shuffle(data)

In [17]:
%%time
for k in data:
    id = index.find(MyKey(k))[0]
    if id is not None:
        pass
        # print(f"found: {id} -> {index.keys[id]}")
    else:
        index.find(k)
        print(f"Key: {k} not found")

print("Done!")

Done!
CPU times: user 2min 58s, sys: 0 ns, total: 2min 58s
Wall time: 2min 58s


In [23]:
len(data) // (180 - 2)

56179

In [22]:
1000000000/1000


1000000.0